In [1]:
#### Import libraries

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
import gensim

In [3]:
import nltk
from nltk.corpus import stopwords
import re
import joblib
from keras.preprocessing.text import Tokenizer

from keras.utils import pad_sequences
from sklearn.preprocessing import LabelEncoder
from keras.layers import Embedding
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [4]:
#### Import dataset 

In [5]:
FoodTweets = pd.read_csv("C:/Users/User/Documents/GitHub/MSC_DA_CA2v4/ProcessFilteredTweets.csv", error_bad_lines=False)

b'Skipping line 1904: expected 6 fields, saw 8\nSkipping line 2003: expected 6 fields, saw 7\nSkipping line 4594: expected 6 fields, saw 7\n'


In [6]:
FoodTweets.head(5)

,column1,column2,column3,column4,column5,column6
0,0,1.678358e+09,Sat May 02 06:26:48 PDT 2009,NO_QUERY,kconnxx,glad to know my family bought tons of gluten f...
1,0,1.827494e+09,Sun May 17 10:49:49 PDT 2009,NO_QUERY,MarieLuv,@pornlaw damn u on the grill when i'm in vegas...
2,0,1.834532e+09,Mon May 18 03:17:33 PDT 2009,NO_QUERY,angelynewolfe,tofu too healthy... searching through the cupb...
3,0,1.963275e+09,Fri May 29 12:28:39 PDT 2009,NO_QUERY,UlyssesAR,looking for food at home!.... nothing
4,0,1.976219e+09,Sat May 30 16:50:48 PDT 2009,NO_QUERY,Kelsayy237,watching something on food network with corey....


In [7]:
#### 1. Rename columns headings

In [8]:
FoodTweets.rename(columns = {'column1':'Sentiment', 'column2':'ids',
                              'column3':'date','column4':'flag','column5':'user','column6':'tweet'}, inplace = True)                                           

In [9]:
FoodTweets.head(5)

,Sentiment,ids,date,flag,user,tweet
0,0,1.678358e+09,Sat May 02 06:26:48 PDT 2009,NO_QUERY,kconnxx,glad to know my family bought tons of gluten f...
1,0,1.827494e+09,Sun May 17 10:49:49 PDT 2009,NO_QUERY,MarieLuv,@pornlaw damn u on the grill when i'm in vegas...
2,0,1.834532e+09,Mon May 18 03:17:33 PDT 2009,NO_QUERY,angelynewolfe,tofu too healthy... searching through the cupb...
3,0,1.963275e+09,Fri May 29 12:28:39 PDT 2009,NO_QUERY,UlyssesAR,looking for food at home!.... nothing
4,0,1.976219e+09,Sat May 30 16:50:48 PDT 2009,NO_QUERY,Kelsayy237,watching something on food network with corey....


In [10]:
FoodTweets['Polarity'] = FoodTweets.loc[:, 'Sentiment']

In [11]:
### 2. Removing the unnessary columns

In [12]:
FoodTweetsa = FoodTweets [['Sentiment','tweet','date','Polarity']]
FoodTweetsa.head(5)

,Sentiment,tweet,date,Polarity
0,0,glad to know my family bought tons of gluten f...,Sat May 02 06:26:48 PDT 2009,0
1,0,@pornlaw damn u on the grill when i'm in vegas...,Sun May 17 10:49:49 PDT 2009,0
2,0,tofu too healthy... searching through the cupb...,Mon May 18 03:17:33 PDT 2009,0
3,0,looking for food at home!.... nothing,Fri May 29 12:28:39 PDT 2009,0
4,0,watching something on food network with corey....,Sat May 30 16:50:48 PDT 2009,0


In [13]:
### 3. Updating the Polarity column with negative , neutral and positive words

In [14]:
FoodTweetsa.Polarity.replace({0:'Negative',4:'Positive'},inplace=True)

C:\Users\User\anaconda3\lib\site-packages\pandas\core\series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [15]:
FoodTweetsa.to_csv('FoodTweets.csv')

In [16]:
### 4. Removing stopwords

In [17]:
stop_words=set(stopwords.words('english'))
#stop_words.remove('not')

In [18]:
corpus=[]
for i in range(0,len(FoodTweets)):
    review=re.sub('@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+',' ',FoodTweetsa['tweet'][i])
    review=review.lower()
    review=review.split()
    review=[word for word in review if not word in stop_words]
    review=' '.join(review)
    corpus.append(review)

In [19]:
FoodTweetsa.tweet = corpus

C:\Users\User\anaconda3\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [20]:
### 6. Creating the Training and testing datasets

In [21]:
from sklearn.model_selection import train_test_split
train_df,test_df=train_test_split(FoodTweetsa,test_size=0.20,random_state=123)

In [22]:
train_df.head()

,Sentiment,tweet,date,Polarity
1369,0,weehee wait get home though food u lookin gooo...,Wed Jun 03 09:08:47 PDT 2009,Negative
1172,0,well right low food budget money,Sat Jun 06 19:15:19 PDT 2009,Negative
3848,0,keep eating junk food,Mon Jun 15 21:08:38 PDT 2009,Negative
2686,0,never eating applebees food unsatisfying stoma...,Wed Jun 17 21:12:03 PDT 2009,Negative
7006,4,chillin family getting tacos ochoa best food w...,Sat May 09 19:05:41 PDT 2009,Positive


In [23]:
test_df.head()

,Sentiment,tweet,date,Polarity
697,0,know gotta get food get work,Sun May 03 17:19:10 PDT 2009,Negative
6875,4,currently looking around food got drunken munc...,Sun Jun 07 16:18:54 PDT 2009,Positive
3508,0,problem away home 4 weeks amp returning sub ze...,Fri Jun 19 17:37:53 PDT 2009,Negative
7386,4,thanks putting us today foodbuzz top 9 guess l...,Fri Jun 05 08:56:19 PDT 2009,Positive
798,0,thinking lunch soon food,Fri May 29 08:07:14 PDT 2009,Negative


In [24]:
### 7. Word2Vec(w2v) 

In [25]:
documents = [text.split() for text in train_df.tweet]

In [26]:
documents = [text.split() for text in train_df.tweet]

In [27]:
w2v_model = gensim.models.word2vec.Word2Vec(vector_size=300, 
                                            window=7, 
                                            min_count=10, 
                                            workers=8)

In [28]:
w2v_model.build_vocab(documents)

In [29]:
vocab_len = len(w2v_model.wv)
print(vocab_len)

921


In [30]:
w2v_model.train(documents, total_examples=len(documents), epochs=30)

(1019208, 1910880)

In [31]:
w2v_model.wv.most_similar("good")

[('times', 0.8372765183448792),
 ('cute', 0.800591230392456),
 ('fun', 0.7911839485168457),
 ('music', 0.7778568267822266),
 ('via', 0.7760693430900574),
 ('friends', 0.7743887901306152),
 ('enjoying', 0.7503080368041992),
 ('company', 0.7415608763694763),
 ('beer', 0.7358059883117676),
 ('played', 0.7268369197845459)]

In [32]:
w2v_model.wv.most_similar("hate")

[('damn', 0.9077363610267639),
 ('wanted', 0.8782784938812256),
 ('ugh', 0.8726919293403625),
 ('something', 0.8604718446731567),
 ('dont', 0.8425645232200623),
 ('mouth', 0.8269162178039551),
 ('cat', 0.8242608308792114),
 ('lmao', 0.8162350654602051),
 ('hell', 0.8154190182685852),
 ('yet', 0.8115626573562622)]

In [33]:
w2v_model.wv.most_similar("great")

[('awesome', 0.9297168254852295),
 ('weather', 0.916863739490509),
 ('lovely', 0.9105941653251648),
 ('music', 0.9074801802635193),
 ('company', 0.8821378946304321),
 ('friends', 0.8689173460006714),
 ('evening', 0.8498674035072327),
 ('beer', 0.8485385775566101),
 ('playing', 0.8465533256530762),
 ('festival', 0.8360029458999634)]

In [34]:
### 8. Tokenisation

In [35]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df.tweet)

In [36]:
tokenizer.word_index

{'food': 1,
 'good': 2,
 'get': 3,
 'amp': 4,
 'eat': 5,
 'go': 6,
 'like': 7,
 'hungry': 8,
 'day': 9,
 'need': 10,
 'going': 11,
 'lol': 12,
 'quot': 13,
 'today': 14,
 'home': 15,
 'love': 16,
 'foods': 17,
 'eating': 18,
 'got': 19,
 'time': 20,
 'great': 21,
 'much': 22,
 'im': 23,
 'want': 24,
 'back': 25,
 'really': 26,
 'work': 27,
 'think': 28,
 'night': 29,
 'chinese': 30,
 'dinner': 31,
 'well': 32,
 'ate': 33,
 'make': 34,
 '2': 35,
 'u': 36,
 'still': 37,
 'lunch': 38,
 'house': 39,
 'feel': 40,
 'oh': 41,
 'one': 42,
 'haha': 43,
 'know': 44,
 'yummy': 45,
 'last': 46,
 'bad': 47,
 'poisoning': 48,
 'miss': 49,
 'better': 50,
 'right': 51,
 'nice': 52,
 'tonight': 53,
 'fun': 54,
 'gonna': 55,
 'though': 56,
 'shopping': 57,
 'way': 58,
 'junk': 59,
 'free': 60,
 'went': 61,
 'see': 62,
 'tomorrow': 63,
 'fast': 64,
 'bring': 65,
 'getting': 66,
 'seafood': 67,
 'would': 68,
 'people': 69,
 'best': 70,
 'awesome': 71,
 'hope': 72,
 'w': 73,
 'sick': 74,
 'thai': 75,
 'tha

In [37]:
vocab_size=len(tokenizer.word_index)+1
vocab_size

11131

In [38]:
X_train = pad_sequences(tokenizer.texts_to_sequences(train_df.tweet), maxlen=300)
X_train

array([[    0,     0,     0, ...,  4360,   206,   649],
       [    0,     0,     0, ...,     1,  1290,   159],
       [    0,     0,     0, ...,    18,    59,     1],
       ...,
       [    0,     0,     0, ...,   598,    36,  2633],
       [    0,     0,     0, ..., 11130,     1,   222],
       [    0,     0,     0, ...,    87,  3997,   236]])

In [39]:
X_test = pad_sequences(tokenizer.texts_to_sequences(test_df.tweet), maxlen=300)
X_test

array([[    0,     0,     0, ...,     1,     3,    27],
       [    0,     0,     0, ...,    36,    24,   659],
       [    0,     0,     0, ...,     1,    47,   634],
       ...,
       [    0,     0,     0, ...,   138, 10515,  2004],
       [    0,     0,     0, ...,  5906,  1520,  1029],
       [    0,     0,     0, ...,    85,   116,     1]])

In [40]:
y_train=train_df.Sentiment
y_train.head()

1369    0
1172    0
3848    0
2686    0
7006    4
Name: Sentiment, dtype: int64

In [41]:
y_test=test_df.Sentiment
y_test.head()

697     0
6875    4
3508    0
7386    4
798     0
Name: Sentiment, dtype: int64

In [42]:
labelencoder = LabelEncoder()
y_train = labelencoder.fit_transform(y_train)
y_test=labelencoder.fit_transform(y_test)

In [43]:
y_train.shape

(6742,)

In [44]:
y_test.shape

(1686,)

In [45]:
embedding_matrix = np.zeros((vocab_size, 300))
for word, i in tokenizer.word_index.items():
  if word in w2v_model.wv:
    embedding_matrix[i] = w2v_model.wv[word]
print(embedding_matrix.shape)

(11131, 300)


In [46]:
embedding_layer = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=300, trainable=False)

In [47]:
model = Sequential()
model.add(embedding_layer)
model.add(Dropout(0.5))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 300)          3339300   
                                                                 
 dropout (Dropout)           (None, 300, 300)          0         
                                                                 
 lstm (LSTM)                 (None, 128)               219648    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 3,559,077
Trainable params: 219,777
Non-trainable params: 3,339,300
_________________________________________________________________


In [48]:
model.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

In [49]:
model_history=model.fit(X_train, y_train,validation_data=(X_test, y_test),batch_size=32,epochs=30,verbose=1)

Epoch 1/30
211/211 [==============================] - 478s 2s/step - loss: 0.6237 - accuracy: 0.6566 - val_loss: 0.5940 - val_accuracy: 0.6898
Epoch 2/30
211/211 [==============================] - 508s 2s/step - loss: 0.6098 - accuracy: 0.6649 - val_loss: 0.6007 - val_accuracy: 0.6821
Epoch 3/30
211/211 [==============================] - 513s 2s/step - loss: 0.5998 - accuracy: 0.6752 - val_loss: 0.5818 - val_accuracy: 0.7005
Epoch 4/30
211/211 [==============================] - 575s 3s/step - loss: 0.5928 - accuracy: 0.6802 - val_loss: 0.5868 - val_accuracy: 0.6762
Epoch 5/30
211/211 [==============================] - 559s 3s/step - loss: 0.5830 - accuracy: 0.6872 - val_loss: 0.5755 - val_accuracy: 0.7040
Epoch 6/30
211/211 [==============================] - 480s 2s/step - loss: 0.5805 - accuracy: 0.6885 - val_loss: 0.5767 - val_accuracy: 0.6975
Epoch 7/30
211/211 [==============================] - 537s 3s/step - loss: 0.5709 - accuracy: 0.6991 - val_loss: 0.5728 - val_accuracy: 0.6892

In [ ]:
# Evaluate the model on the test set for 1 month
_, accuracy_1m = model.evaluate(X_test, y_test)


In [ ]:
# Train the model for 3 months
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=7, batch_size=32)

Epoch 1/90
211/211 [==============================] - 515s 2s/step - loss: 0.3413 - accuracy: 0.8410 - val_loss: 0.7760 - val_accuracy: 0.6643
Epoch 2/90
211/211 [==============================] - 558s 3s/step - loss: 0.3316 - accuracy: 0.8540 - val_loss: 0.7716 - val_accuracy: 0.6625
Epoch 3/90
211/211 [==============================] - 514s 2s/step - loss: 0.3090 - accuracy: 0.8653 - val_loss: 0.8153 - val_accuracy: 0.6643
Epoch 4/90
211/211 [==============================] - 514s 2s/step - loss: 0.2993 - accuracy: 0.8659 - val_loss: 0.8062 - val_accuracy: 0.6595
Epoch 5/90
211/211 [==============================] - 486s 2s/step - loss: 0.2956 - accuracy: 0.8701 - val_loss: 0.8084 - val_accuracy: 0.6643
Epoch 6/90
211/211 [==============================] - 502s 2s/step - loss: 0.2836 - accuracy: 0.8776 - val_loss: 0.8448 - val_accuracy: 0.6625
Epoch 7/90
211/211 [==============================] - 493s 2s/step - loss: 0.2792 - accuracy: 0.8769 - val_loss: 0.8511 - val_accuracy: 0.6613

In [ ]:
# Evaluate the model on the test set for 3 months
_, accuracy_3m = model.evaluate(X_test, y_test)

In [ ]:
Train the model for 1 week
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=7, batch_size=32)

In [ ]:
# Evaluate the model on the test set for 1 week
_, accuracy_1w = model.evaluate(X_test, y_test)


In [ ]:
# Create the Dash app
app = dash.Dash(__name__)

In [ ]:
# Define the layout of the app
app.layout = html.Div(children=[
    html.H1('Sentiment Analysis Dashboard'),
    html.Div(children=[
        html.H3('Accuracy'),
        html.Div(children=[
            html.Div('1 week:'),
            html.Div(f'{accuracy_1w:.4f}'),
        ]),
        html.Div(children=[
            html.Div('1 month:'),
            html.Div(f'{accuracy_1m:.4f}'),
        ]),
        html.Div(children=[
            html.Div('3 months:'),
            html.Div(f'{accuracy_3m:.4f}'),
        ]),
    ]),
    dcc.Graph(
        id='accuracy-graph',
        figure={
            'data': [
                {'x': ['1 week', '1 month', '3 months'], 'y': [accuracy_1w, accuracy_1m, accuracy_3m], 'type': 'bar', 'name': 'Accuracy'},
            ],
            'layout': {
                'title': 'Sentiment Analysis Accuracy for Different Time Periods',
                'yaxis': {'title': 'Accuracy'},
            }
        }
    )
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)